# VAST Database Steps

## Install and Import required libraries

In [ ]:
!pip install findspark

In [ ]:

import json

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
import os
from pyspark.sql.utils import AnalysisException
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import pandas as pd

from IPython.display import display, HTML

## Load Demo State and define variables

In [ ]:
# Read in the Demo Variables and values.
with open("demo_state.json", "r") as f:
    data = json.load(f)
# Dynamically create python variables needed for the Demo.
for key, value in data.items():
    globals()[key] = value

VASTDB_JARS_DIR = '/opt/vastdb_spark/'
# Get a comma-separated string of all JARs in the directory
jars = ",".join([os.path.join(VASTDB_JARS_DIR, f) for f in os.listdir(VASTDB_JARS_DIR) if f.endswith(".jar")])
jars_path = ":".join([os.path.join(VASTDB_JARS_DIR, f) for f in os.listdir(VASTDB_JARS_DIR) if f.endswith(".jar")])
DB_ENDPOINT = f"http://{vip_pool_ip}"

## Define Functions

In [ ]:
def sqlcmd(sqlString):
    try:
        df_result = spark.sql(sqlString)
    except AnalysisException as e:
        print(f"AnalysisException occurred: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")
    return df_result    

## Define Spark Instance Configuration

In [ ]:
conf = (
     pyspark.SparkConf()
        .set("spark.driver.extraClassPath", jars_path)
        .set("spark.driver.userClassPathFirst", "true")
        .set("spark.executor.cores","1")
        .set("spark.executor.userClassPathFirst", "true")
        .set("spark.jars", jars)
    # VAST Natural Database Configuration Options
        .set("spark.ndb.access_key_id",S3_ACCESS_KEY)
        .set("spark.ndb.secret_access_key",S3_SECRET_KEY)
        .set("spark.ndb.endpoint",DB_ENDPOINT)
        .set("spark.ndb.dynamic_filter_compaction_threshold", 100)
        .set("spark.ndb.dynamic_filtering_wait_timeout", 2)
        .set("spark.ndb.parallel_import", "true")
        .set("spark.ndb.retry_max_count",3)
        .set("spark.ndb.retry_sleep_duration",1)
        .set("spark.sql.catalog.ndb", "spark.sql.catalog.ndb.VastCatalog")
        .set("spark.sql.extensions","ndb.NDBSparkSessionExtension")
        .set("spark.ndb.num_of_splits",8)
        .set("spark.ndb.num_of_sub_splits",8)
        .set("spark.ndb.rowgroups_per_subsplit",1)
        .set("spark.ndb.query_data_rows_per_split",4000000)
        .set("spark.network.timeout", "3600s")
        .set("spark.port.maxRetries", 30)
        .set("spark.rpc.askTimeout", "3000s")
        .set("spark.rpc.io.connectionTimeout", "3000s")
   #    .set("spark.rpc.numRetries", 15)
   #    .set("spark.rpc.retry.wait", "3000s") 
        .set("spark.shuffle.io.connectionTimeout", "3000s")
        .set("spark.shuffle.push.finalize.timeout", "3000s")
        .set("spark.speculation", "false")
        .set("spark.sql.execution.arrow.pyspark.enabled", "true")
        .set("spark.sql.readSideCharPadding","False")
        .set("spark.task.maxFailures", "1")
        .set("spark.task.reaper.pollingInterval", "3000s")  
        .setAppName('VASTDB')
        .setMaster("local[2]")
)
if SparkSession.getActiveSession() is not None:
    # Stop the SparkSession
    SparkSession.getActiveSession().stop()
    print("SparkSession stopped.")
else:
    print("No active SparkSession found.")

## Start Spark Instance

In [ ]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
# Get the SparkConf from the SparkContext
conf = spark.sparkContext.getConf()
print("Spark Running")
spark_version = spark.sparkContext.version
print(f"Spark version: {spark_version}")

## Create the "Database / Schema" for the Demo

In [ ]:
sql_text = f"CREATE DATABASE IF NOT EXISTS `ndb`.`{vastdb_bucket}`.`{demo_suffix}`"
sqlcmd(sql_text)


In [ ]:
sqlContext = f"USE `ndb`.`{vastdb_bucket}`.`{demo_suffix}`;"
sqlcmd(sqlContext)

In [ ]:
#
# Create the Table to store the events for IDrive.
# 
sqlCMD = """CREATE TABLE IF NOT EXISTS idrive (
    eventSource STRING,
    awsRegion STRING,
    lastModified TIMESTAMP,
    eventName STRING,
    userIdentity_principalId STRING,
    requestParameters_sourceIPAddress STRING,
    responseElements_x_amz_request_id STRING,
    responseElements_x_amz_id_2 STRING,
    s3_configurationId STRING,
    s3_bucket_name STRING,
    s3_bucket_ownerIdentity_principalId STRING,
    s3_bucket_arn STRING,
    s3_object_key STRING,
    s3_object_ext STRING,
    s3_object_size BIGINT,
    s3_object_sequencer STRING,
    s3_tags MAP<STRING, STRING>);"""
sqlcmd(sqlCMD)



In [ ]:
#
# Create the Table to store the S3 tags for IDrive.
# 
sqlCMD = """CREATE TABLE IF NOT EXISTS tag_metadata (
    awsRegion STRING,
    s3_bucket_name STRING,
    s3_object_key STRING,
    key STRING,
    string_value STRING,
    int_value INT,
    float_value FLOAT, 
    bool_value BOOLEAN
  
);"""

sqlcmd(sqlCMD)



In [ ]:
#
# List the tables that are now in the "database"
#
sqlCMD = "SHOW TABLES;"
df_result = sqlcmd(sqlCMD)

# Data is returned in a Spark Dataframe
df_result.show(truncate=False)

## Create SQL for TABLE Create

In [ ]:
sqlCMD = "SHOW CREATE TABLE idrive;"
df=sqlcmd(sqlCMD)
pdf = df.select("createtab_stmt").toPandas()
pdf['createtab_stmt'] = pdf['createtab_stmt'].str.replace('\n', '<br>')
display(HTML(pdf.to_html(escape=False, index=False)))

## Query Object Metadata Values

In [ ]:
df = spark.sql("select * from tag_metadata LIMIT 20;")
pdf = df.toPandas()
pdf['int_value'] = pdf['int_value'].fillna(0).astype(int) # The toPandas() method doesnt capture the datatype correctly.
display(HTML(pdf.to_html(index=False)))

## Query IDrive Table

In [ ]:
sqltxt = """
SELECT
    eventSource,
    awsRegion,
    lastModified, 
    eventName,
    userIdentity_principalId,
    requestParameters_sourceIPAddress,
    responseElements_x_amz_request_id,
    responseElements_x_amz_id_2,
    s3_configurationId,
    s3_bucket_name,
    s3_bucket_ownerIdentity_principalId,
    s3_bucket_arn,
    s3_object_key,
    s3_object_ext,
    s3_object_size,
    s3_object_sequencer
FROM idrive LIMIT 20;
"""  # LIMIT 20 , WHERE s3_object_key = 'site1/g34zduhh.xlsx
df = spark.sql(sqltxt) 
if isinstance(df, pyspark.sql.dataframe.DataFrame) and not df.isEmpty():
    pdf = df.toPandas()
  #  pdf['int_value'] = pdf['int_value'].fillna(0).astype(int) # The toPandas() method doesnt capture the datatype correctly.
    display(HTML(pdf.to_html(index=False)))
else:
    print("No records returned from SQL Query.")

> Switch to the S3 Events - Object Source Notebook to create S3 Object Events.

## SQL Aggregate Functions

In [ ]:
#
# Query how many EXCEL Files are in the idrive table.
#
print("EXCEL Files loaded:")
sqltxt = """
 SELECT COUNT(*) AS TOTAL 
  FROM idrive 
  WHERE s3_object_ext = 'xlsx';
"""
df = spark.sql(sqltxt)
pdf = df.toPandas()
display(HTML(pdf.to_html(index=False)))

#
# Query the total number of Files that are in the idrive table.
#
print("TOTAL Files loaded:")
sqltxt = """
 SELECT COUNT(*) AS TOTAL 
  FROM idrive;
"""
df = spark.sql(sqltxt)
pdf = df.toPandas()
display(HTML(pdf.to_html(index=False)))

In [ ]:
#
# Report on the number of files for each unique file extension. 
#
sqltxt = """
SELECT 
    s3_object_ext, 
    COUNT(*) AS count, 
    SUM(s3_object_size) AS total_size
FROM idrive
GROUP BY s3_object_ext
ORDER BY count DESC;
"""
df = spark.sql(sqltxt)
pdf = df.toPandas()
display(HTML(pdf.to_html(index=False)))

In [ ]:
#
# Report the Unique S3 user tags and how many files have each tag.
#
sqltxt = """
SELECT 
    key,
    COUNT(*) AS record_count
FROM 
    tag_metadata
GROUP BY 
    key
ORDER BY 
    record_count DESC;
"""
df = spark.sql(sqltxt)
if isinstance(df, pyspark.sql.dataframe.DataFrame) and not df.isEmpty():
    pdf = df.toPandas()
    display(HTML(pdf.to_html(index=False)))
else:
    print("No records returned from SQL Query.")

## Inner Join

In [ ]:
#
# Query the S3 User tag metadata for each idrive (File) record that has a "docx" extension.
#

sqltxt= """
  SELECT
    idrive.lastModified, 
    idrive.eventName,
    idrive.s3_bucket_name,
    idrive.s3_object_key,
    idrive.s3_object_ext,
    idrive.s3_object_size,
    tag_metadata.key,
    tag_metadata.string_value,
    tag_metadata.int_value,
    tag_metadata.float_value,
    tag_metadata.bool_value
FROM idrive
JOIN tag_metadata
ON idrive.awsRegion = tag_metadata.awsRegion
   AND idrive.s3_bucket_name = tag_metadata.s3_bucket_name
   AND idrive.s3_object_key = tag_metadata.s3_object_key
WHERE idrive.s3_object_ext = 'docx' 
ORDER BY idrive.s3_object_key
LIMIT 40; 
"""
df = spark.sql(sqltxt)
if isinstance(df, pyspark.sql.dataframe.DataFrame) and not df.isEmpty():
    pdf = df.toPandas()
    pdf['int_value'] = pdf['int_value'].fillna(0).astype(int) # The toPandas() method doesnt capture the datatype correctly.
    display(HTML(pdf.to_html(index=False)))
else:
    print("No records returned from SQL Query.")

#### Query for S3 User Tag : "ASCII" set to False

In [ ]:

df=[]
sqltxt= """
SELECT
    idrive.lastModified,  
    idrive.eventName,
    idrive.s3_bucket_name,
    idrive.s3_object_key,
    idrive.s3_object_ext,
    idrive.s3_object_size,
    tag_metadata.key,
    tag_metadata.string_value,
    tag_metadata.int_value,
    tag_metadata.float_value,
    tag_metadata.bool_value
FROM idrive
JOIN tag_metadata
  ON idrive.awsRegion = tag_metadata.awsRegion
 AND idrive.s3_bucket_name = tag_metadata.s3_bucket_name
 AND idrive.s3_object_key = tag_metadata.s3_object_key
WHERE tag_metadata.key = 'ASCII'
  AND tag_metadata.bool_value IS FALSE
LIMIT 40;
"""
df = spark.sql(sqltxt)
if isinstance(df, pyspark.sql.dataframe.DataFrame) and not df.isEmpty():
    pdf = df.toPandas()
    pdf['int_value'] = pdf['int_value'].fillna(0).astype(int) # The toPandas() method doesnt capture the datatype correctly.
    display(HTML(pdf.to_html(index=False)))
    print(pdf.shape)
else:
    print("No records returned from SQL Query.")


## SQL Table Aliasing

In [ ]:
#
# Query for S3 objects associated with the "EFS"  Project, and have a PSI value between 2500 AND 3500
#
sqltxt = """
SELECT 
    idrive.lastModified,  
    idrive.eventName,
    idrive.s3_bucket_name,
    idrive.s3_object_key,
    idrive.s3_object_ext,
    idrive.s3_object_size,
    tm_project.key AS project_key,
    tm_project.string_value AS project_value,
    tm_psi.key AS psi_key,
    tm_psi.int_value AS psi_value
FROM idrive
JOIN tag_metadata tm_project
  ON idrive.awsRegion = tm_project.awsRegion
 AND idrive.s3_bucket_name = tm_project.s3_bucket_name
 AND idrive.s3_object_key = tm_project.s3_object_key
JOIN tag_metadata tm_psi
  ON idrive.awsRegion = tm_psi.awsRegion
 AND idrive.s3_bucket_name = tm_psi.s3_bucket_name
 AND idrive.s3_object_key = tm_psi.s3_object_key
WHERE tm_project.key = 'Project'
  AND tm_project.string_value = 'EFS'
  AND tm_psi.key = 'PSI'
  AND tm_psi.int_value BETWEEN 2500 AND 3500
  ORDER BY idrive.s3_object_ext;
"""
df = spark.sql(sqltxt)
if isinstance(df, pyspark.sql.dataframe.DataFrame) and not df.isEmpty():
    pdf = df.toPandas()
    pdf['psi_value'] = pdf['psi_value'].fillna(0).astype(int) # The toPandas() method doesnt capture the datatype correctly.
    display(HTML(pdf.to_html(index=False)))
    print(pdf.shape)
else:
    print("No records returned from SQL Query.")

In [ ]:
sqltxt = """
SELECT 
    idrive.lastModified,  
    idrive.eventName,
    idrive.s3_bucket_name,
    idrive.s3_object_key,
    idrive.s3_object_ext,
    idrive.s3_object_size,
    tm_project.key AS project_key,
    tm_project.string_value AS project_value,
    tm_psi.key AS psi_key,
    tm_psi.int_value AS psi_value
FROM idrive
JOIN tag_metadata tm_project
  ON idrive.awsRegion = tm_project.awsRegion
 AND idrive.s3_bucket_name = tm_project.s3_bucket_name
 AND idrive.s3_object_key = tm_project.s3_object_key
JOIN tag_metadata tm_psi
  ON idrive.awsRegion = tm_psi.awsRegion
 AND idrive.s3_bucket_name = tm_psi.s3_bucket_name
 AND idrive.s3_object_key = tm_psi.s3_object_key
WHERE tm_project.key = 'Project'
  AND tm_project.string_value = 'GoA'
  AND tm_psi.key = 'PSI'
  AND tm_psi.int_value BETWEEN 2500 AND 3500
ORDER BY idrive.s3_object_ext;
"""
df = spark.sql(sqltxt)
if isinstance(df, pyspark.sql.dataframe.DataFrame) and not df.isEmpty():
    pdf = df.toPandas()
    pdf['psi_value'] = pdf['psi_value'].fillna(0).astype(int) # The toPandas() method doesnt capture the datatype correctly.
    display(HTML(pdf.to_html(index=False)))
    print(pdf.shape)
else:
    print("No records returned from SQL Query.")

## SQL Table management  

In [ ]:
df = spark.sql("DELETE FROM tag_metadata;")
df.show()

In [ ]:
df = spark.sql("DELETE FROM idrive;")
df.show()

In [ ]:
sqltxt = """
DROP TABLE idrive;
DROP TABLE tag_metadata;
"""
df = spark.sql(sqltxt)
df.show()